In [1]:
import cufflinks as cf
from plotly.offline import plot, iplot
import pandas as pd
import numpy as np

In [2]:
cf.go_offline()

<p>VISUALIZAÇÃO E GERAÇÃO DE INSIGHTS</p>

FUNCIONÁRIOS

In [3]:
# Importando o arquivo
df_employers = pd.read_parquet('gold/employers.parquet')

## Perguntas aos dados

- Dados de Contratação e Demissão:
	- Taxa de rotatividade.
	- Tempo médio de permanência.
- Relacionamento com Cargos:
	- Distribuição de funcionários por cargo.
	- Relação entre cargo e nível.

In [5]:
# Dados de contratação e Demissão (Taxa de Rotatividade)
#df_employers.info()
df_employers.data_de_demissao = pd.to_datetime(df_employers.data_de_demissao)
num_funcionarios_que_deixaram = df_employers['data_de_demissao'].notnull().sum()



num_funcionarios_medio = df_employers['id_rh'].nunique()  # Número médio de funcionários

taxa_rotatividade = (num_funcionarios_que_deixaram / num_funcionarios_medio) * 100

print(f'Taxa de Rotatividade: {taxa_rotatividade:.2f}%')

Taxa de Rotatividade: 19.11%
